# Predict Images
Given a model and images the model is applied and the result is added to the metadata

In [ ]:
!pip3 install tensorflow==2.4.0 wget==3.2

In [ ]:
import wget
wget.download(
    'https://raw.githubusercontent.com/'
    'elyra-ai/component-library/master/claimed_utils.py'
)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from claimed_utils import unzip
import os
import os.path
import glob
import pandas as pd
import numpy as np

In [ ]:
# @dependency codait_utils.ipynb
# @dependency model
# @dependency image data
# @dependency metadata
# @param target_column Column name containing the target/prediction value
# (the real measured value)
# @param image_column Column name containing the image file name
# @param image_shape Target image shape (the model expects)
# @param prediction_column Column name under which the
# prediction has to be stored
# @param metadata Csv file name of the meta data
# @param model zip file name
# @param data zip file name

In [ ]:
target_column = os.environ.get('target_column', 'target')
image_column = os.environ.get('image_column', 'filename')
image_shape = os.environ.get('image_shape', '400,400')
prediction_column = os.environ.get('prediction_column', 'prediction')
metadata = os.environ.get('metadata', 'metadata.csv')
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')

In [ ]:
unzip('.', model_zip)
unzip('.', data_zip)

In [ ]:
model = keras.models.load_model('model')

In [ ]:
classes = glob.glob("data/*")
classes = map(lambda s: s.split('/')[1], classes)
classes = list(classes)
classes

In [ ]:
df = pd.read_csv(metadata)

In [ ]:
df

In [ ]:
dataset = []

for i, row in df.iterrows():
    target_size = 'dummy'  # make the compiler happy
    exec('target_size = (' + image_shape + ')')
    image = tf.keras.preprocessing.image.load_img(
        'data/' + row[target_column] + '/' + row[image_column],
        target_size=target_size
    )
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = model.predict(input_arr)
    prediction = predictions[0]
    prediction = classes[np.argmax(prediction)]
    dataset.append(prediction)

In [ ]:
dataset = np.array(dataset)

In [ ]:
df = pd.read_csv(metadata)

In [ ]:
df[prediction_column] = dataset.tolist()

In [ ]:
df

In [ ]:
df.to_csv(metadata, index=False)